In [ ]:
import pandas as pd
from bokeh.charts import output_notebook, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.layouts import row
from bokeh.plotting import figure

output_notebook() 


In [ ]:
dataset = pd.read_csv("data.csv",index_col=0,parse_dates=True)


rich_weekday = lambda c: -1 if c["holiday"] else c["weekday"]

dataset.workingday = dataset.workingday==1
dataset.holiday = dataset.holiday==1
dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["season_s"] = dataset.season.map({1:"Printemps",2:"Ete",3:"Automne",4:"Hiver"})
dataset["weather_s"] = dataset.weather.map({1:"clear",2:"cloudy",3:"light rain",4:"crappy"})
dataset["rich_weekday"] = dataset.apply( rich_weekday, axis=1)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
label = dataset["registered"] 
del dset["count"], dset["registered"], dset["casual"]
dset["hour"] =  dset.index.hour
dset["weekday"] = dset.index.weekday
#dset["hour_weekend"] = dset["hour"] * dataset.weekday.map(lambda i: 0 if (i in [6,7]) else 1)
#dset["hour_workweek"] = dset["hour"] * dataset.weekday.map(lambda i: 1 if (i in [6,7]) else 0)   
dset["h_wd"] = dset.weekday*100+dataset.hour

dset_scaled = preprocessing.scale(dset)
pca = PCA(n_components="mle")

dt = DecisionTreeRegressor(max_depth=25)
clf = AdaBoostRegressor(n_estimators=100,base_estimator=dt)
#clf = GradientBoostingRegressor(n_estimators=300,max_depth=4) 
#clf = Ridge(normalize=True, alpha=.1, tol=1e-15)
estimators = [('reduce_dim', pca), ('clf',clf)]
pipe = Pipeline(estimators)
#scores = cross_val_score(clf, dset, label,cv=20,verbose=1,scoring="neg_mean_absolute_error",n_jobs=6)
scores = cross_val_score(clf, dset, label,cv=5,verbose=1,scoring="neg_mean_squared_error",n_jobs=6)                          
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print scores

In [ ]:
estimator = pipe.fit(dset,label)
full_dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
full_dset["hour"] =  dset.index.hour
full_dset["weekday"] = dset.index.weekday
full_dset["predicted"] = estimator.predict(dset)

In [ ]:
col = "predicted"
gg =  full_dset.groupby(["weekday","hour"])[col].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y=col,color="weekday")

show(b)

In [ ]:
from sklearn.model_selection import GridSearchCV
print clf.get_params()
#param_grid = [{'learning_rate':[1],'base_estimator__max_depth':[2,5,10],'n_estimators':[10,100,500,1000,5000]}]
param_grid = [{'learning_rate':[1],'max_depth':[2,5,10,20,30],'n_estimators':[10,100,500,1000,5000]}]
gscv = GridSearchCV(
    estimator = clf,
    param_grid=param_grid,
    scoring="r2",
    cv=5,
    verbose=2,
    n_jobs=6
)

f = gscv.fit(dset, label)

print f.best_estimator_
print f.best_score_

In [ ]:
from sklearn import svm
from sklearn import preprocessing
dset_scaled = preprocessing.scale(dset)
clf = svm.SVR(kernel='rbf',C=1,verbose=True, cache_size=1000)
#clf = svm.SVR(C=1e3,verbose=True)
scores = cross_val_score(clf, dset, label,cv=10,verbose=3,scoring="neg_median_absolute_error",n_jobs=6)
scores.mean()  
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
cross_val_score?

In [ ]:
dataset["count"].describe()

In [ ]:
from tpot import TPOTRegressor
pipeline_optimizer = TPOTRegressor(generations=5, population_size=20, num_cv_folds=5, random_state=42, verbosity=2)
#(fits by default on lean squared errror)
label = dataset["registered"] 
dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
del dset["count"], dset["registered"], dset["casual"]
dset["hour"] =  dset.index.hour
dset["weekday"] = dset.index.weekday
#dset["hour_weekend"] = dset["hour"] * dataset.weekday.map(lambda i: 0 if (i in [6,7]) else 1)
#dset["hour_workweek"] = dset["hour"] * dataset.weekday.map(lambda i: 1 if (i in [6,7]) else 0)   
dset["h_wd"] = dset.weekday*100+dataset.hour

pipeline_optimizer.fit(dset, label)
print(pipeline_optimizer.score(dset, label))
pipeline_optimizer.export('tpot_exported_pipeline.py')

In [ ]:
dset